In [112]:
# imports
from imgaug import augmenters as iaa
import imgaug as ia
import os
import numpy as np
import cv2
import glob
from PIL import Image
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import time 


In [113]:
# seed for ia randomizer
ia.seed(1)


In [114]:
# fetch certain file types
images_names = []
ignored = 0

for fname in os.listdir('images'):
    if fname.endswith('.jpg'):
        images_names.append(fname)
    else:
        ignored += 1

print("found", len(images_names), ".jpg files")
print("ignored", ignored, "other file types")


found 9 .jpg files
ignored 9 other file types


In [115]:
# extract only bounding box information from .txt file (see format below)
def extract_bounding_boxes(file):
    bounding_boxes = []
    with open(file) as f:
        boxes = f.readlines()
        for box in boxes:
            box_arr = box.split(' ')
            bounding_boxes.append(
                [box_arr[1], box_arr[2], box_arr[3], box_arr[4].strip("\n")])
    return bounding_boxes

# extract only the classification label from .txt file (see format below)


def extract_classification_labels(file):
    class_labels = []
    with open(file) as f:
        labels = f.readlines()
        for label in labels:
            lab_arr = label.split(' ')
            class_labels.append(lab_arr[0])
    return class_labels


Bounding box .txt files should look as follows:

3 0.297656 0.615278 0.118750 0.205556   
3 0.622656 0.374306 0.056250 0.093056

Where the first number represents a classification label

In [116]:
# put images in np-array from memory & put corresponding bounding boxes in array in order
# bounding_box_images contains tuples in which the first element corresponds to the image and the second element corresponds to the bounding boxes details as an array
bounding_box_images = []
classification_labels = []

for image_name in images_names:
    # obtain image
    location = "images/"+image_name
    image = cv2.imread(location)

    # obtain bounding boxes
    loc = os.path.splitext(location)[0] + '.txt'
    bounding_boxes = extract_bounding_boxes(loc)

    # put data in tuple and append to array
    bounding_box_images.append((image, bounding_boxes))

    # save classification labels for future use
    classification_labels.append(extract_classification_labels(loc))


print("images found with bounding box(es):", len(bounding_box_images))
print("classifications found for boxes:", len(classification_labels))
print(classification_labels)


images found with bounding box(es): 9
classifications found for boxes: 9
[['0'], ['3'], ['3', '3'], ['3', '0'], ['3'], ['0'], ['0'], ['3', '3'], ['3', '3', '3', '3', '3']]


In [117]:

# convert bounding box of format [x1, y1, x2, y2] to yolo box of format [X, Y, W, H]
def bbox2yolobox(img_shape, box):
    box = [box.x1, box.y1, box.x2, box.y2]
    dw = 1./img_shape[1]
    dh = 1./img_shape[0]
    x = (box[0] + box[2])/2.0
    y = (box[1] + box[3])/2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    x = round(x*dw, 6)
    w = round(w*dw, 6)
    y = round(y*dh, 6)
    h = round(h*dh, 6)
    return [x,y,w,h]

# convert yolo box of format[X, Y, W, H] to bounding box of format [x1, y1, x2, y2]
def yolobbox2bbox(img_shape, box):
    height = img_shape[0]
    width = img_shape[1]
    box = [float(val) for val in box]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    x1, y1 = round((x-w/2)*width, 6), round((y-h/2)*height,6)
    x2, y2 = round((x+w/2)*width, 6), round((y+h/2)*height, 6)
    
    return [x1, y1, x2, y2]


In [118]:
# augmented_bb_images folder path
path = 'augmented_bb_images/'


In [119]:
# ONLY EXECUTE THIS CELL BLOCK IF YOU WANT THE CURRENT AUGMENTED IMAGES FOLDER TO BE DELETED!!!

# empty augmented_images folder
if( input("Do you want to delete the current augmented_bb_images folder? (y/n)") == "y"):
    for f in os.listdir(path):
        os.remove(os.path.join(path, f))
    print("deleted augmented_bb_images folder...")
else:
    print("appending new data to existing folder...")

deleted augmented_bb_images folder...


In [120]:
def save_data(data, identifier):
    for idx, images in enumerate(data):

        # save augmented image
    
        name =  f"AUG-{identifier}-" + images_names[idx] 
        
        cv2.imwrite(os.path.join(path, name), images[0]) 

        # save bounding boxes in yolo format to .txt file
        boxes = images[1]
        name =  path + f"AUG-{identifier}-" + os.path.splitext(images_names[idx])[0] + '.txt'

        f = open(name, 'w+')
        
        for idx2, box in enumerate(boxes):
            # print(augmented_bb_images[idx][1])
            box_string = ""
            for item in box:
                box_string += str(item) + " "
            box_string += "\n"
            try:
                f.write(box_string)
            except Exception as ex:
                print("something went wrong when saving bounding boxes for image:", name)
                print(ex)

        f.close()
        


In [121]:
def augment(seq):
    start = time.time()
    # perform image & bounding box augmentation on all images
    augmented_bb_images = []
    for bb_img in bounding_box_images:

        # convert yolo format to bounding box format
        bbs_formatted = [yolobbox2bbox(bb_img[0].shape, bb) for bb in bb_img[1]]

        # define bbs as required for the imgaug library (BoundingBoxesOnImage type)
        bbs = BoundingBoxesOnImage([
            BoundingBox(x1=float(bb[0]), y1=float(bb[1]), x2=float(bb[2]), y2=float(bb[3])) for bb in bbs_formatted
        ], shape=bb_img[0].shape)

        # Augment BBs and images.
        aug_img, aug_bbs = seq(image=bb_img[0], bounding_boxes=bbs)

        # convert augmented bounding boxes to yolo format
        aug_bbs = [bbox2yolobox(bb_img[0].shape, bb)
                for bb in aug_bbs.bounding_boxes]

        # place augmented image with corresponding augmented boxes in array
        aug_bb_img = [aug_img, aug_bbs]

        # append array to all augmented data
        augmented_bb_images.append(aug_bb_img)

    end = time.time()
    elapsed = round((end-start),3)
    print(f'succesfully augmented {len(augmented_bb_images)} images with bounding boxes in {elapsed} sec')

    # add classification label to each box
    for idx, images in enumerate(augmented_bb_images):
        for idx2, box in enumerate(images[1]):
            label = classification_labels[idx][idx2]
            label_box = [label]
            for b in box:
                label_box.append(b)
            augmented_bb_images[idx][1][idx2] = label_box
            
    return augmented_bb_images



In [122]:
# sequential operations

# vertical flip
seq_vertical = iaa.Sequential([
    iaa.Flipud(1)
])

# horizontal flip
seq_horizontal = iaa.Sequential([
    iaa.Fliplr(1)
])

# vertical & horizontal flip
seq_vert_hor = iaa.Sequential([
    iaa.Flipud(1),
    iaa.Fliplr(1)
])

# positive saturation (2x multiplier)
seq_saturate = iaa.Sequential([
    iaa.MultiplySaturation(2, 2)
])


In [123]:
# augmentation 1
aug1 = augment(seq_vertical)
save_data(aug1, "VERTICAL")

# augmentation 2
aug2 = augment(seq_horizontal)
save_data(aug2, "HORIZONTAL")

# augmentation 3
aug3 = augment(seq_vert_hor)
save_data(aug3, "VERTICAL_HORIZONTAL")

# augmentation 4
# aug4 = augment(seq_saturate)
# save_data(aug4, "SATURATED")

succesfully augmented 9 images with bounding boxes in 0.007 sec
succesfully augmented 9 images with bounding boxes in 0.021 sec
succesfully augmented 9 images with bounding boxes in 0.024 sec


AssertionError: Expected `from_colorspaces` to be either an iterable of strings or a single string. Got type: int.